<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#-Комментарий-ревьюера" data-toc-modified-id="-Комментарий-ревьюера-4"><span class="toc-item-num">4&nbsp;&nbsp;</span> Комментарий ревьюера</a></span></li></ul></div>

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score

try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv('toxic_comments.csv')
data_initial = data
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [3]:
data.describe()

,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


В данных присутствуют 159571 сообщение, примерно каждое 10-е из них является токсичным.

Разделим данные на тренировочную и тестовую выборки в соотношении 4:1, предварительно отделив целевой признак от текста.

In [4]:
X, y = data.drop(columns='toxic'), data['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

def check_size(name, sample):
    print(f'Размер {name} выборки датасета: {len(sample)}')
check_size('обучающей', y_train)
check_size('тестовой', y_test)

Размер обучающей выборки датасета: 127656
Размер тестовой выборки датасета: 31915


Создадим корпуса для обеих выборок. Переведём их в нижний регистр и в кодировку Unicode (U).

In [5]:
corpus_train = X_train['text'].str.lower().values.astype('U')
corpus_test = X_test['text'].str.lower().values.astype('U')

Для расчета TD-IDF создадим счётчик, указав в нём стоп-слова, обучим его на тренировочной выборке и подсчитаем TD-IDF для обеих выборок.

In [6]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

tf_idf_train = count_tf_idf.fit_transform(corpus_train)
tf_idf_test = count_tf_idf.transform(corpus_test)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aleksandr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Обучение

Построим модель случайного леса с подбором параметров через RandomizedSearch.

In [7]:
%%time

RF_params = {'n_estimators': range(1, 51, 10),
             'max_depth': range(1, 15),
             'class_weight': [None, 'balanced'],
             'min_samples_split': range(2, 10)}

RS_random_forest = RandomizedSearchCV(RandomForestClassifier(random_state=1),
                                      param_distributions=RF_params,
                                      cv=5, scoring='f1',
                                      n_iter=20, random_state=1)

RS_random_forest.fit(tf_idf_train, y_train)

F1_random_forest = RS_random_forest.best_score_

print(f'Лучшая F1 случайного леса: {F1_random_forest}')
print(f'Параметры модели случайного леса: {RS_random_forest.best_params_}')

C:\anaconda3\envs\practicum\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\anaconda3\envs\practicum\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\anaconda3\envs\practicum\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\anaconda3\envs\practicum\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\anaconda3\envs\practicum\lib\site-packages\sklearn\metrics\classification.py:

Лучшая F1 случайного леса: 0.3364269977194401
Параметры модели случайного леса: {'n_estimators': 41, 'min_samples_split': 6, 'max_depth': 9, 'class_weight': 'balanced'}
Wall time: 4min 1s


Построим модель логистической регрессии с подбором параметров через RandomizedSearch.

In [8]:
%%time

LR_params = {'C': np.logspace(-3, 3, num=7),
             'tol': np.logspace(-7, -2, num=6)}

RS_log_reg = RandomizedSearchCV(LogisticRegression(random_state=1),
                                      param_distributions=LR_params,
                                      cv=5, scoring='f1', 
                                      n_iter=20, random_state=1)

RS_log_reg.fit(tf_idf_train, y_train)

F1_log_reg = RS_log_reg.best_score_

print(f'Лучшая F1 логистической регрессии: {F1_log_reg}')
print(f'Параметры модели логистической регрессии: {RS_log_reg.best_params_}')

C:\anaconda3\envs\practicum\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\anaconda3\envs\practicum\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Лучшая F1 логистической регрессии: 0.7631271352533507
Параметры модели логистической регрессии: {'tol': 0.01, 'C': 10.0}
Wall time: 3min 25s


При подборе парметров на кросс-валидации с использованием моделей случайного леса и логистической регрессии наилучший результат метрики F1 по валидационным выборкам (0.76 против 0.33) показала логистическая регрессия. Ее и будем проверять на тестовой выборке.

## Выводы

Проверим качество логистической регрессии на тестовой выборке.

In [9]:
prediction = RS_log_reg.predict(tf_idf_test)
print(f'F1 модели логистической регрессии на тестовой выборке: {f1_score(y_test, prediction)}')

F1 модели логистической регрессии на тестовой выборке: 0.7853116994022205


В ходе проекта были обучены две модели для классификации комментариев на позитивные и негативные. Расчитав величину TF-IDF для корпуса текстов и обучив на нем модели, выяснили, что наилучший результат метрики F1 в данной задаче бинарной классификации показала логистическая регрессия с параметрами tol: 0.01 и C: 10. Проверив ее на тестовой выборке и получив показатель F1=0.78, убедились, что она соответсвует требуемому уровню качества (F1 не меньше 75).